In [ ]:
import pandas as pd

diana = pd.read_csv('../../engines/DIANA_targets.txt', sep = '\t')

targetscan = pd.read_csv('../../engines/Targetscan_targets.txt', sep = '\t')

mtb = pd.read_csv('../../engines/MTB_targets_25.csv')

tarbase = pd.read_csv('../../engines/TarBase_v9.tsv', sep = '\t')

C:\Users\aoviedomadr\AppData\Local\Temp\ipykernel_9416\4243239025.py:9: DtypeWarning: Columns (5,14,15,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  tarbase = pd.read_csv('TarBase_v9.tsv', sep = '\t')


In [8]:
tarbase = tarbase[tarbase['mirna_name'].str.contains('hsa')]
tarbase = tarbase[['mirna_name', 'gene_id']]
tarbase.columns = ['miRNA', 'Gene ID']
tarbase = tarbase[tarbase['miRNA'].str.lower().str.startswith('hsa')]
tarbase['miRNA'] = tarbase['miRNA'].str.lower()
tarbase.drop_duplicates(inplace=True)

In [9]:
tscan = targetscan[['Gene ID', 'miRNA']]
diana = diana[['mirna', 'ensembl_gene_id']]
mtb = mtb[['miRNA', 'Target Gene']]

In [ ]:
gtf = pd.read_csv(
    '../../engines/gencode.v47.basic.annotation.gtf',
    sep='\t',
    comment='#',
    header=None,
    names=[
        'seqname', 'source', 'feature', 'start', 'end',
        'score', 'strand', 'frame', 'attribute'
    ]
)

gtf['gene_id'] = gtf['attribute'].str.extract(r'gene_id "([^"]+)"')
gtf['gene_name'] = gtf['attribute'].str.extract(r'gene_name "([^"]+)"')
gtf = gtf[['gene_id', 'gene_name']]

In [ ]:
gene_map = gtf[['gene_name', 'gene_id']].drop_duplicates()
mtb_with_ids = mtb.merge(gene_map, left_on='Target Gene', right_on='gene_name', how='left')

In [12]:
mtb_with_ids = mtb_with_ids[['miRNA', 'gene_id']]
tscan = tscan[['miRNA', 'Gene ID']]

In [13]:
mtb_with_ids.columns = ['miRNA', 'Gene ID']
diana.columns = ['miRNA', 'Gene ID']
tscan.columns = ['miRNA', 'Gene ID']

In [14]:
mtb_with_ids = mtb_with_ids[mtb_with_ids['miRNA'].str.lower().str.startswith('hsa')]

In [15]:
mtb_with_ids = mtb_with_ids[mtb_with_ids['miRNA'].str.lower().str.startswith('hsa')]
diana = diana[diana['miRNA'].str.lower().str.startswith('hsa')]
tscan = tscan[tscan['miRNA'].str.lower().str.startswith('hsa')]
mtb_with_ids['miRNA'] = mtb_with_ids['miRNA'].str.lower()
diana['miRNA'] = diana['miRNA'].str.lower()
tscan['miRNA'] = tscan['miRNA'].str.lower()

In [16]:
mtb_with_ids['Gene ID'] = mtb_with_ids['Gene ID'].str.split('.').str[0]
tscan['Gene ID'] = tscan['Gene ID'].str.split('.').str[0]
diana['Gene ID'] = diana['Gene ID'].str.split('.').str[0]

In [17]:
tscan.drop_duplicates(inplace=True)
mtb_with_ids.drop_duplicates(inplace=True)
diana.drop_duplicates(inplace = True)

In [18]:
tscan_set = set([tuple(row) for row in tscan.values])
mtb_with_ids_set = set([tuple(row) for row in mtb_with_ids.values])
diana_set = set([tuple(row) for row in diana.values])
tarbase_set = set([tuple(row) for row in tarbase.values])

In [15]:
def matching_percentage(df_main, df_other):
    # Crear conjuntos de tuplas para comparación fila a fila
    main_set = set([tuple(row) for row in df_main.values])
    other_set = set([tuple(row) for row in df_other.values])
    
    # Número de coincidencias
    matches = main_set & other_set
    return len(matches) / len(df_main) * 100, len(matches)

# Comparaciones
p12, matches12 = matching_percentage(mtb_with_ids, tscan)
print(f"mtb vs tscan: {p12:.4f}%")
print(f"mtb vs tscan:", matches12)
p13, matches13 = matching_percentage(mtb_with_ids, diana)
print(f"mtb vs diana: {p13:.4f}%")
print(f"mtb vs diana:", matches13)
p21, matches21 = matching_percentage(diana, mtb_with_ids)
print(f"diana vs mtb: {p21:.4f}%")
print(f"diana vs mtb:", matches21)
p23, matches23 = matching_percentage(diana, tscan)
print(f"diana vs tscan: {p23:.4f}%")
print(f"diana vs tscan:", matches23)
p31, matches31 = matching_percentage(tscan, mtb_with_ids)
print(f"tscan vs mtb: {p31:.4f}%")
print(f"tscan vs mtb:", matches31)
p32, matches32 = matching_percentage(tscan, diana)
print(f"tscan vs diana: {p32:.4f}%")
print(f"tscan vs diana:", matches32)
p41, matches41 = matching_percentage(tarbase, mtb_with_ids)
print(f"tarbase vs mtb: {p41:.4f}%")
print(f"tarbase vs mtb:", matches41)
p42, matches42 = matching_percentage(tarbase, tscan)
print(f"tarbase vs tscan: {p42:.4f}%")
print(f"tarbase vs tscan:", matches42)
p43, matches43 = matching_percentage(tarbase, diana)
print(f"tarbase vs diana: {p43:.4f}%")
print(f"tarbase vs diana:", matches43)
p14, matches14 = matching_percentage(mtb_with_ids, tarbase)
print(f"mtb vs tarbase: {p14:.4f}%")
print(f"mtb vs tarbase:", matches14)
p24, matches24 = matching_percentage(tscan, tarbase)
print(f"tscan vs tarbase: {p24:.4f}%")
print(f"tscan vs tarbase:", matches24)
p34, matches34 = matching_percentage(diana, tarbase)
print(f"diana vs tarbase: {p34:.4f}%")
print(f"diana vs tarbase:", matches34)

KeyboardInterrupt: 